In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 🧬 Solow-Romer Hybrid Growth Model

This model integrates key features from two major growth theories:
1.  **Solow Model:** Emphasizes the role of **capital accumulation ($K$)** subject to diminishing returns and exogenous technological progress.
2.  **Romer Model:** Focuses on **endogenous technological progress ($A$)** driven by investment in research and development (R&D), often linked to the number of researchers ($L_A$).

By combining these, the hybrid model can explain both the transitional dynamics (convergence towards a balanced growth path, similar to Solow) and the long-run sustained growth driven by endogenous innovation (similar to Romer).

# ⚙️ Model Equations

The core equations of this simplified hybrid model are:

1.  **Output Production:** Output ($Y$) is produced using capital ($K$) and labor allocated to goods production ($L_Y$), augmented by the level of technology/ideas ($A$). Assumes Cobb-Douglas form:
    $$Y_t = K_t^{\alpha} (A_t L_{Y,t})^{1 - \alpha}$$

2.  **Capital Accumulation:** The capital stock changes based on savings ($s$) out of output and depreciation ($\delta_K$).
    $$\dot{K}_t = s Y_t - \delta_K K_t$$
    *(Discrete time version: $K_{t+1} = K_t + s Y_t - \delta_K K_t$)*

3.  **Idea/Technology Accumulation:** The stock of ideas ($A$) grows based on the number of workers allocated to R&D ($L_A$) and the existing stock of ideas. $\delta_A$ represents the productivity of R&D workers.
    $$\dot{A}_t = \delta_A L_{A,t} A_t$$
    *(Discrete time version: $A_{t+1} = A_t + \delta_A L_{A,t} A_t = A_t (1 + \delta_A L_{A,t})$)*

4.  **Labor Allocation:** The total labor force ($L$) is split between producing goods ($L_Y$) and producing ideas ($L_A$). We assume a constant fraction $l_A$ works in R&D. The total labor force grows at an exogenous rate $n$.
    $$L_A = l_A L$$
    $$L_Y = (1 - l_A) L$$
    $$L_{t+1} = L_t (1 + n)$$

**Key Difference from Basic Solow:** Technological growth ($g_A = \dot{A}/A = \delta_A L_A$) is *endogenous* – it depends on the allocation of labor to R&D ($L_A$).

# 📈 Balanced Growth Path (BGP)

Unlike the basic Solow model where growth eventually stops without exogenous technological progress, the Solow-Romer hybrid model can exhibit a **Balanced Growth Path (BGP)** where key variables grow at constant rates indefinitely.

On the BGP:
* The growth rate of technology ($A$) is constant: $g_A = \delta_A L_A = \delta_A l_A L$. *Self-Correction: If L is growing, $g_A$ is not constant unless $\delta_A$ depends on L. A common Romer formulation is $\dot{A} = \delta_A L_A A^{\phi}$ with $\phi < 1$ or $\dot{A}/A = \delta_A L_A^{\lambda} A^{\phi-1}$. The simplest version used here, $\dot{A}/A = \delta_A L_A$, implies $g_A$ grows if $L_A$ grows.* Let's assume $L$ is constant ($n=0$) for a simple BGP analysis or use a slightly different idea equation for stable BGP growth rate. *Modification: Let's stick to the provided $\dot{A}/A = \delta_A L_A$ but assume $L$ is constant ($n=0$) for the BGP discussion and simulation simplification.*
* Output per worker ($Y/L$) and Capital per worker ($K/L$) grow at the rate $g_A$.
* Total Output ($Y$) and Total Capital ($K$) grow at the rate $g_A$ (since $n=0$).

The simulation below tracks the levels of $K$, $A$, and $Y$ over time. We will also plot growth rates to observe convergence towards a BGP (if parameters allow).

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def solow_romer_hybrid_model(s=0.2, delta_k=0.05, alpha=0.33,
                             delta_A=0.001, l_A=0.1, L0=100, n=0.0, # Added n, default 0 for BGP
                             A0=1.0, K0=10.0, T=100):
    """
    Simulates a hybrid Solow-Romer endogenous growth model.

    Args:
        s (float): Savings rate.
        delta_k (float): Depreciation rate of capital.
        alpha (float): Capital share in output production.
        delta_A (float): Productivity parameter for R&D.
        l_A (float): Fraction of labor force in R&D (0 < l_A < 1).
        L0 (float): Initial total labor force.
        n (float): Growth rate of the total labor force L.
        A0 (float): Initial level of technology/ideas.
        K0 (float): Initial capital stock.
        T (int): Time Horizon (number of periods).
    """
    # Ensure T is an integer
    T = int(T)
    if T < 5: T = 5

    # Input validation
    alpha = np.clip(alpha, 0.01, 0.99)
    s = np.clip(s, 0.01, 0.99)
    delta_k = max(delta_k, 0)
    l_A = np.clip(l_A, 0.01, 0.99)
    delta_A = max(delta_A, 0)
    n = max(n, 0) # Allow zero population growth
    L0 = max(L0, 1)
    A0 = max(A0, 0.1)
    K0 = max(K0, 0.1)


    # Arrays to store results (size T+1 to include initial values)
    time = np.arange(T + 1)
    A = np.zeros(T + 1)
    K = np.zeros(T + 1)
    L = np.zeros(T + 1)
    Y = np.zeros(T + 1)
    L_A = np.zeros(T + 1) # Labor in R&D
    L_Y = np.zeros(T + 1) # Labor in Goods Production
    g_A = np.zeros(T)     # Growth rate of A (length T)
    g_K = np.zeros(T)     # Growth rate of K
    g_L = np.zeros(T)     # Growth rate of L
    g_Y = np.zeros(T)     # Growth rate of Y

    # Initial conditions
    A[0] = A0
    K[0] = K0
    L[0] = L0

    # --- Simulation Loop (t=0 to T-1 to calculate state for t+1) ---
    for t in range(T):
        # Labor allocation for period t
        L_A[t] = l_A * L[t]
        L_Y[t] = (1 - l_A) * L[t]

        # Production in period t (handle potential zero inputs)
        if K[t] <= 0 or A[t] <= 0 or L_Y[t] <= 0:
             Y[t] = 0
        else:
             Y[t] = K[t]**alpha * (A[t] * L_Y[t])**(1 - alpha)

        # Calculate next period's state variables
        # Technology growth: A_dot = delta_A * L_A * A
        A[t+1] = A[t] * (1 + delta_A * L_A[t])

        # Capital accumulation: K_dot = sY - delta_k K
        K[t+1] = K[t] + s * Y[t] - delta_k * K[t]
        K[t+1] = max(K[t+1], 0) # Ensure capital doesn't go negative

        # Labor force growth
        L[t+1] = L[t] * (1 + n)

        # Store growth rates (from t to t+1)
        if A[t] > 0: g_A[t] = (A[t+1] / A[t]) - 1
        if K[t] > 0: g_K[t] = (K[t+1] / K[t]) - 1
        if L[t] > 0: g_L[t] = (L[t+1] / L[t]) - 1 # Should be equal to n
        # Calculate Y[T] for the last period
        if t == T-1:
             L_A[T] = l_A * L[T]
             L_Y[T] = (1 - l_A) * L[T]
             if K[T] <= 0 or A[T] <= 0 or L_Y[T] <= 0:
                  Y[T] = 0
             else:
                  Y[T] = K[T]**alpha * (A[T] * L_Y[T])**(1 - alpha)
        if Y[t] > 0: g_Y[t] = (Y[t+1] / Y[t]) - 1


    # --- Calculate Per Capita Variables ---
    Y_per_L = Y / L
    K_per_L = K / L

    # --- Plotting ---
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.ravel()

    # Plot 0: Levels (Log Scale)
    axes[0].plot(time, Y, label='Total Output (Y)', color='black', lw=2)
    axes[0].plot(time, K, label='Total Capital (K)', color='blue', linestyle='--')
    axes[0].plot(time, A, label='Technology (A)', color='red', linestyle='-.')
    axes[0].plot(time, L, label='Labor (L)', color='green', linestyle=':')
    axes[0].set_yscale('log')
    axes[0].set_title('Levels of Key Variables (Log Scale)')
    axes[0].set_xlabel("Time (t)")
    axes[0].set_ylabel("Level (Log Scale)")
    axes[0].legend(fontsize='small')
    axes[0].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 1: Growth Rates
    plot_time_g = time[:-1] # Time for growth rates (length T)
    axes[1].plot(plot_time_g, g_Y*100, label='Output Growth (gY)', color='black', lw=2)
    axes[1].plot(plot_time_g, g_K*100, label='Capital Growth (gK)', color='blue', linestyle='--')
    axes[1].plot(plot_time_g, g_A*100, label='Tech Growth (gA)', color='red', linestyle='-.')
    axes[1].plot(plot_time_g, g_L*100, label='Labor Growth (gL=n)', color='green', linestyle=':')
    axes[1].set_title('Annual Growth Rates')
    axes[1].set_xlabel("Time (t)")
    axes[1].set_ylabel("Growth Rate (%)")
    axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1f}%'))
    axes[1].legend(fontsize='small')
    axes[1].grid(True, linestyle='--', alpha=0.6)

    # Plot 2: Per Capita Variables (Log Scale)
    axes[2].plot(time, Y_per_L, label='Output per Worker (Y/L)', color='black', lw=2)
    axes[2].plot(time, K_per_L, label='Capital per Worker (K/L)', color='blue', linestyle='--')
    axes[2].plot(time, A, label='Technology (A)', color='red', linestyle='-.') # A is often plotted with per-capita
    axes[2].set_yscale('log')
    axes[2].set_title('Per Worker Variables (Log Scale)')
    axes[2].set_xlabel("Time (t)")
    axes[2].set_ylabel("Level per Worker (Log Scale)")
    axes[2].legend(fontsize='small')
    axes[2].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 3: Growth Rate of Technology (gA) vs R&D Labor (LA)
    axes[3].plot(time[:-1], L_A[:-1], label='R&D Labor (LA)', color='purple', linestyle=':')
    ax3b = axes[3].twinx() # Secondary axis for gA
    ax3b.plot(time[:-1], g_A*100, label='Tech Growth (gA)', color='red', linestyle='-.')
    axes[3].set_title('Technology Growth and R&D Labor')
    axes[3].set_xlabel("Time (t)")
    axes[3].set_ylabel("R&D Labor (LA)", color='purple')
    ax3b.set_ylabel("Tech Growth Rate (gA %)", color='red')
    axes[3].tick_params(axis='y', labelcolor='purple')
    ax3b.tick_params(axis='y', labelcolor='red')
    ax3b.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1f}%'))
    # Combine legends
    lines, labels = axes[3].get_legend_handles_labels()
    lines2, labels2 = ax3b.get_legend_handles_labels()
    axes[3].legend(lines + lines2, labels + labels2, loc='best', fontsize='small')
    axes[3].grid(True, linestyle='--', alpha=0.6)


    fig.suptitle(f"Solow-Romer Hybrid Model (n={n:.1%})", fontsize=16, y=1.03)
    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.show()

    # --- Display Final Growth Rates ---
    final_gY = g_Y[-1] if T > 0 else np.nan
    final_gA = g_A[-1] if T > 0 else np.nan
    final_gK = g_K[-1] if T > 0 else np.nan
    final_gL = g_L[-1] if T > 0 else np.nan

    results_md = f"""
    ### 📈 Approximate Growth Rates in Final Period (t={T-1}):

    * **Output Growth (gY):** {final_gY:.2%}
    * **Capital Growth (gK):** {final_gK:.2%}
    * **Labor Growth (gL):** {final_gL:.2%} (should equal n)
    * **Technology Growth (gA):** {final_gA:.2%} (driven by $\\delta_A L_A$)
    * **Output per Worker Growth (gY - gL):** {final_gY - final_gL:.2%} (approaches gA on BGP if n=0)
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    solow_romer_hybrid_model,
    s=FloatSlider(value=0.2, min=0.05, max=0.5, step=0.01, description='Savings Rate (s):', style=style, layout=layout, readout_format='.2f'),
    delta_k=FloatSlider(value=0.05, min=0.01, max=0.15, step=0.005, description='Capital Deprec (δ_k):', style=style, layout=layout, readout_format='.3f'),
    alpha=FloatSlider(value=0.33, min=0.1, max=0.8, step=0.01, description='Capital Share (α):', style=style, layout=layout, readout_format='.2f'),
    delta_A=FloatSlider(value=0.001, min=0.0, max=0.01, step=0.0001, description='R&D Productivity (δ_A):', style=style, layout=layout, readout_format='.4f'),
    l_A=FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='R&D Labor Share (l_A):', style=style, layout=layout, readout_format='.2f'),
    L0=FloatSlider(value=100, min=10, max=1000, step=10, description='Initial Labor (L0):', style=style, layout=layout, readout_format='.0f'),
    n=FloatSlider(value=0.00, min=0.0, max=0.05, step=0.002, description='Labor Growth (n):', style=style, layout=layout, readout_format='.1%'),
    A0=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Initial Tech (A0):', style=style, layout=layout, readout_format='.1f'),
    K0=FloatSlider(value=10.0, min=1, max=100.0, step=1, description='Initial Capital (K0):', style=style, layout=layout, readout_format='.0f'),
    T=IntSlider(value=100, min=20, max=300, step=10, description='Time Periods (T):', style=style, layout=layout)
);


interactive(children=(FloatSlider(value=0.2, description='Savings Rate (s):', layout=Layout(width='95%'), max=…

# 🧬 Solow-Romer Hybrid Model

This model combines:
- Capital accumulation (from Solow)
- Endogenous technology (from Romer)


# Equations

**Output:**
\[
Y = K^\alpha (A L_Y)^{1 - \alpha}
\]

**Capital accumulation:**
\[
\frac{dK}{dt} = s Y - \delta K
\]

**Ideas:**
\[
\frac{dA}{dt} = \delta_A L_A A
\]

Labor is split:
- \( L = L_Y + L_A \)
- Total labor grows exogenously at rate \( n \)


# 🧠 Key Insight

> In this model, **both A(t)** and **K(t)** grow — and so does Y(t).  
> There is **no steady-state in levels** — the whole system grows over time.

It captures:
- **Transition** → Solow style
- **Sustained growth** → Romer style